In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from PIL import Image
from torchvision import transforms
import numpy as np
import os
from sklearn.impute import SimpleImputer
from typing import Optional, Tuple, Dict
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [2]:
class DataMushroom(Dataset):
    def __init__(self, file_csv_x, file_csv_y=None):
        """
        Инициализация датасета грибов
        Args:
            file_csv_x: DataFrame с входными признаками
            file_csv_y: DataFrame с целевыми переменными (опционально)
        """
        self.file_csv_x = file_csv_x.copy().astype(int, errors='ignore') 
        self.file_csv_y = file_csv_y.copy().astype(int, errors='ignore') if file_csv_y is not None else pd.DataFrame()
        self.encoders_x = {}
        self.encoders_y = {}
        
        # Сохраняем служебные столбцы отдельно
        self.observation_ids = self.file_csv_x['observationID'].copy()
        self.filenames = self.file_csv_x['filename'].copy()
        
        # Удаляем служебные столбцы перед обработкой
        self.file_csv_x = self.file_csv_x.drop(['observationID', 'filename'], axis=1)
        
        # Сохраняем исходные метки до токенизации
        if not self.file_csv_y.empty:
            self.original_labels = self.file_csv_y.copy()
        
        self._preprocess_data()
        
    def _preprocess_data(self):
        """Предобработка данных: токенизация и заполнение пропусков"""
        self.token_x, self.token_y = self._tokenize_features()
        self.token_x, self.token_y = self._fill_missing_values()

        
    def _tokenize_features(self):
        """Токенизация категориальных признаков"""
        # Токенизация X
        for column in self.file_csv_x.select_dtypes(include=['object']).columns:
            self.encoders_x[column] = LabelEncoder()
            self.file_csv_x[column] = self.encoders_x[column].fit_transform(
                self.file_csv_x[column].astype(str)
            )
            
        # Токенизация y (всех столбцов)
        if not self.file_csv_y.empty:
            for column in self.file_csv_y.columns:
                self.encoders_y[column] = LabelEncoder()
                self.file_csv_y[column] = self.encoders_y[column].fit_transform(
                    self.file_csv_y[column].astype(str)
                )
        
        return self.file_csv_x, self.file_csv_y
    
    def _fill_missing_values(self):
        """Заполнение пропущенных значений с помощью RandomForest"""
        def fill_df_missing_values(df):
            for column in df.columns:
                if df[column].isnull().any():
                    mask = df[column].isnull()
                    non_null_data = df[~mask].copy()  # Только строки без NaN в целевой колонке
                    
                    if len(non_null_data) > 0:
                        # Удаляем целевую колонку из признаков и заполняем NaN в признаках
                        features = non_null_data.drop(column, axis=1)
                        
                        # Применяем импутер к признакам, чтобы не было NaN
                        imputer = SimpleImputer(strategy='mean')
                        features_imputed = imputer.fit_transform(features)
                        
                        target = non_null_data[column]
                        
                        rf = RandomForestRegressor(n_estimators=100, random_state=42)
                        rf.fit(features_imputed, target)
                        
                        # Данные, где нужно предсказать пропуски
                        missing_data = df.loc[mask].drop(column, axis=1)
                        
                        # Применяем тот же импутер к данным с пропусками
                        missing_data_imputed = imputer.transform(missing_data)
                        
                        # Предсказываем и заполняем пропуски
                        df.loc[mask, column] = rf.predict(missing_data_imputed)
    
            return df
    
        self.token_x = fill_df_missing_values(self.token_x)
        if not self.token_y.empty:
            self.token_y = fill_df_missing_values(self.token_y)
        
        return self.token_x, self.token_y
    
    def __len__(self):
        return len(self.observation_ids.unique())
    
    def __getitem__(self, idx):
        """
        Получение элемента датасета по индексу
        Args:
            idx: индекс элемента
        Returns:
            dict: словарь с данными элемента
        """
        # Получаем все уникальные observation_ids и их индексы
        unique_obs_ids = self.observation_ids.unique()
        
        # Получаем все строки данных для этого observation_id
        obs_mask = self.observation_ids == unique_obs_ids[idx]
        x_data = self.token_x[obs_mask]
        
        # Получаем все связанные изображения
        patch_filenames = self.filenames[obs_mask].values
        
        max_patches = 10
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomVerticalFlip(p=0.5),
            transforms.RandomRotation(degrees=30),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        
        images_tensor = torch.zeros(max_patches, 3, 224, 224)
        
        for i in range(min(max_patches, len(patch_filenames))):
            img = Image.open(f'E:/jupyter/Грибы/images/images/train/300p/{patch_filenames[i]}')
            images_tensor[i] = transform(img)
        
        # Преобразуем x_data в матрицу нужной формы
        num_features = x_data.shape[1]  # Количество признаков
        x_tensor = torch.tensor(x_data.values[0], dtype=torch.float32)
        
        sample = {
            'x_data': x_tensor,
            'images_tensor': images_tensor
        }
        
        if not self.token_y.empty:
            # Получаем все токенизированные метки
            y_data = self.token_y[obs_mask].values
            y_tensor = torch.tensor(y_data[0], dtype=torch.int64)
            sample['y_data'] = y_tensor
            
        return sample

In [7]:
# Загрузка данных
df = pd.read_csv('E:\jupyter\Грибы\metadata\FungiTastic-FewShot\FungiTastic-FewShot-Train.csv')
# Выбор входных параметров X
X = df[['habitat', 'substrate', 'elevation', 'landcover', 'biogeographicalRegion', 'observationID','filename']]
# Выбор выходных параметров y 
y = df[['kingdom', 'phylum', 'class','order','category_id',]]
data = DataMushroom(X,y)
print(data)
# Создаем загрузчик данных
train_loader = DataLoader(data, batch_size=2, shuffle=False)

In [8]:
for c in train_loader:
    x = c['x_data']
    y = c['y_data']
    print(y)




tensor([[   1,    1,   11,   26, 1582],
        [   2,    8,   21,  101, 1746]])
tensor([[  1,   1,  24,  81, 324],
        [  1,   1,   1,   9, 147]])
tensor([[   1,    1,    9,   90, 1215],
        [   1,    1,   27,   36, 1959]])
tensor([[   1,    1,   16,   78, 1188],
        [   1,    1,   16,   57, 1295]])
tensor([[   1,    1,    9,   17,  787],
        [   1,    1,   16,   57, 1694]])
tensor([[   1,    1,   16,  104, 1569],
        [   1,    1,    9,   90,  952]])
tensor([[   1,    1,   17,   95, 1328],
        [   1,    1,   17,   24, 1246]])
tensor([[  0,   9,  23,  79, 827],
        [  1,   1,  16,  18, 728]])
tensor([[   1,    1,    1,    9, 1236],
        [   1,    1,    9,    1, 2316]])
tensor([[   1,    2,    0,   21,  596],
        [   1,    1,   32,  123, 1810]])
tensor([[   1,    1,   17,   50, 1019],
        [   1,    1,   17,   50, 1009]])
tensor([[   1,    2,   30,  115, 1533],
        [   1,    1,    9,   90,  552]])
tensor([[   1,    1,    9,   90, 2224],
        

KeyboardInterrupt: 

In [9]:
unique_counts = {}
for c in data:
    y = c['y_data']
    # Обрабатываем все столбцы кроме 5 (индекс 4)
    for col in range(y.shape[0]):
        if col == 4:  # Пропускаем 5 столбец
            continue
        unique_vals = torch.unique(y[col])
        if col not in unique_counts:
            unique_counts[col] = set()
        unique_counts[col].update(unique_vals.tolist())
        
for col, vals in unique_counts.items():
    print(f"Столбец {col}: {len(vals)} уникальных значений")

Столбец 0: 3 уникальных значений
Столбец 1: 11 уникальных значений
Столбец 2: 33 уникальных значений
Столбец 3: 124 уникальных значений
Столбец 4: 2427 уникальных значений


KeyboardInterrupt: 

In [ ]:
for epoch in range(10):
    running_loss = 0.0
    for d in train_loader:
        optimizer.zero_grad()
        img = d['images_tensor']
        x_data = d['x_data']
        y_data = d['y_data']

        # Визуализация только первого изображения
        plt.figure(figsize=(5, 5))
        img_to_show = img[0].cpu()  # Берем первое изображение
        if img_to_show.dim() == 4:
            img_to_show = img_to_show.squeeze(0)  # Убираем batch dimension
        if img_to_show.dim() == 3 and img_to_show.size(0) == 3:  # Проверяем RGB
            img_to_show = img_to_show.permute(1, 2, 0)  # Меняем порядок каналов
        plt.imshow(img_to_show.numpy())
        plt.axis('off')
        plt.show()

        output, ouputs = model(img, x_data)
        
        weights = [1/3, 1/11, 1/33, 1/2427]
        total_loss = 0
        for i, (level_reg, level_der) in enumerate(zip(reg, der)):
            preds = ouputs[level_der]
            loss = criterion(preds, y_data.T[level_reg].long())
            
            if i == len(reg) - 1:
                _, topk = preds.topk(10, dim=1)
                correct = topk.eq(y_data.T[level_reg].view(-1,1).expand_as(topk))
                topk_acc = correct.sum().item() / len(y_data)
                print(f'Loss: {loss.item():.4f}, Top-10 Acc: {topk_acc:.4f}')
            else:
                print(f'Loss: {loss.item():.4f}')
                
            total_loss += loss * weights[level_reg]
            
        total_loss.backward()
        optimizer.step()
        print(f'Total Loss: {total_loss.item():.4f}')

In [ ]:
# Создаем словарь
my_dict = {}

# Добавляем несколько значений к одному ключу (используем список)
my_dict['key'] = ['value1']
my_dict['key'].append('value2')  # Добавляем второе значение

# Альтернативный способ инициализации
my_dict = {'key': ['value1', 'value2']}

# Или через setdefault
my_dict.setdefault('key', []).append('value3')


In [ ]:
reg = {}
for epoch in range(10):
    running_loss = 0.0
    for i, d in enumerate(train_loader):
        optimizer.zero_grad()
        img = d['images_tensor'].to(device)
        x_data = d['x_data'].to(device)
        y_data = d['y_data'].to(device)
        
        # Визуализация только первого изображения
        plt.figure(figsize=(5, 5))
        img_to_show = img[0][0].cpu()  # Берем первое изображение
        print(img_to_show.shape)
        if img_to_show.dim() == 4:
            img_to_show = img_to_show.squeeze(0)  # Убираем batch dimension
        if img_to_show.dim() == 3 and img_to_show.size(0) == 3:  # Проверяем RGB
            img_to_show = img_to_show.permute(1, 2, 0)  # Меняем порядок каналов
        plt.imshow(img_to_show.numpy())
        plt.axis('off')
        plt.show()
        
        output,ouputs = model(img, x_data)
        weights = [1/3, 1/11, 1/33,1/67]
        total_loss = 0
        total_loss = 0
        preds = ouputs[3]
        loss = criterion(preds, y_data.T[3].long())
        total_loss += loss
        reg['Simple_loss'].append(total_loss)
        print(loss)

In [ ]:
plt.figure(figsize=(10, 5))

for key, values in reg.items():
    plt.plot([v.item() for v in values], label=key)

plt.title('Losses')
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.legend()
plt.grid()
plt.show()


In [ ]:
reg['Simple_loss'] = []  # Очистка значений ключа 'Simple_loss' (оставляем пустой список)

